In [1]:
import spotipy 
import pandas as pd 
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np

import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import IFrame

<a id="functions"></a>
## 1. Functions

In [2]:
def load_songs(fp:[str]=None):
    if not fp:
        print("No filepath provided")
        return None
    try:
        return pd.read_csv(fp, sep=";", error_bad_lines=False)
    except FileNotFoundError:
        print("File not found.")
        return None

In [9]:
def load_pickle(fp:[str] = None): 
    if not fp:
        print("no filepath provided.")
        return None
    try: 
        with open(fp, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")
        return None

In [3]:
def user_input_get_features(this_df):
    audio_feat = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                  'liveness', 'valence', 'tempo', 'time_signature']
    user_input = input("What song are you looking for? ")
    song_hits = list(this_df[this_df["name"].str.lower()==user_input.lower()]["artist"])
    if song_hits:
        if len(song_hits) == 1:
            numpy_lst = []
            for a_f in audio_feat:
                numpy_lst.append(float(this_df[this_df["name"].str.lower()==user_input.lower()][a_f]))
            return np.array(numpy_lst).reshape(1,-1)
        else:
            print(f'Following artists found for "{user_input}":\n ')
            c = 0
            for a in song_hits:
                c += 1
                print(f" + {c}: {a}")
            print(" -------\n+ S: Search alternative on Spotify")
            user_choice = 241
            while 0 > int(user_choice) or int(user_choice) > len(song_hits)+1:
                user_choice = input(f"Please specify your entry (1 to {len(song_hits)}): ")
                if user_choice.lower() == "s":
                    return get_unknown_song_array(user_input)
                if user_choice.isnumeric() == False:
                    print("please check entry.")
                    user_choice = len(song_hits)+2
                if 0 > int(user_choice) > len(song_hits)+1:
                    print("please check entry.")
            this_df_index = list(this_df[this_df["name"].str.lower()==user_input.lower()].index)[int(user_choice)-1]
            # print(df_.iloc[df_index]["danceability"])
            numpy_lst = []
            for a_f in audio_feat:
                numpy_lst.append(float(this_df.iloc[this_df_index][a_f]))
            return np.array(numpy_lst).reshape(1,-1)
    else:
        print("Song not found in local database, searching on Spotify.")
        return get_unknown_song_array(user_input)

In [5]:
def get_unknown_song_array(this_song:[str]=None):
    if not this_song:
        print("no song provided.")
        return None
    x=sp.search(this_song, limit=15)
    if not x["tracks"]["items"]:
        print("song is not on spotify.")
        return None
    if len(x["tracks"]["items"]) == 1:
        pass
    else:
        artists_of_this_song = []
        for _ in range(len(x["tracks"]["items"])):
            artists_of_this_song.append(x["tracks"]["items"][_]["artists"][0]["name"])
        # artists_of_this_song = sorted(set(artists_of_this_song), key=lambda x: x.lower())
        print(f'Following artists found for "{this_song}":\n ')
        c = 0
        for a in artists_of_this_song:
            c += 1
            print(f" + {c}: {a}")
        user_choice = 241
        while 0 > int(user_choice) or int(user_choice) > len(artists_of_this_song) + 1:
            user_choice = input(f"Please specify your entry (1 to {len(artists_of_this_song)}): ")
            if user_choice.isnumeric() == False:
                print("please check entry.")
                user_choice = len(artists_of_this_song) + 2   # +2 to create a wrong int and run another while loop 
            if 0 > int(user_choice) > len(song_hits) + 1:
                print("please check entry.")
        # print((x["tracks"]["items"][int(user_choice)-1]["artists"][0]["name"]))
        song_info=sp.audio_features([x["tracks"]["items"][int(user_choice)-1]["id"]])
        remove_list = ['mode', 'duration_ms', 'id', 'type', 'uri', 'track_href', 'analysis_url'] #remove features not used in the model
        [song_info[0].pop(key) for key in remove_list] 
        song_list=list(song_info[0].values())
        song_list
        song_array = np.array(song_list).reshape(1, -1)
        song_array
        return song_array

In [51]:
def get_random_song_cluster(this_df, this_song_array):
    X = song_df.drop(['id', 'length','name', 'album','artist','release_date', 'type', 'mode', 'popularity', "cluster"], axis=1) 
    scaler = load_pickle("Model/scaler.pickle")
    kmeans = load_pickle("Model/kmeans_10.pickle")
    scaled_song = scaler.transform(this_song_array)
    cluster_pred = kmeans.predict(scaled_song)[0]
    print("cluster:", cluster_pred)
    index_song = this_df[this_df['cluster'] == cluster_pred].sample().index[0]
    id_song = this_df.iloc[index_song]['id']
    print("get_r", id_song)
    return print(sp.track(id_song)["name"], "by", sp.track(id_song)["artists"][0]["name"])

In [40]:
def add_cluster_column_to_song_df(this_df):
    #define features
    X = this_df.drop(['id', 'length','name', 'album','artist','release_date', 'type', 'mode', 'popularity'], axis=1) 
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
    
    kmeans = KMeans(n_clusters=10, random_state=42)
    kmeans.fit(X_scaled_df)
    
    clusters = kmeans.predict(X_scaled_df)
    #clusters
    pd.Series(clusters).value_counts().sort_index()
    this_df["cluster"] = clusters
    return this_df

In [35]:
song_df = add_cluster_column_to_song_df(song_df)

In [41]:
song_df

,id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,...,liveness,loudness,speechiness,valence,key,mode,tempo,type,time_signature,cluster
0,30QR0ndUdiiMQMA9g1PGCm,"...And to Those I Love, Thanks for Sticking Ar...",Stop Staring at the Shadows,$uicideboy$,2020-02-14,168489,85,0.792,0.124000,0.511,...,0.1400,-6.876,0.0409,0.1110,2,1,113.983,audio_features,4,1
1,496GZe7MxLloHgUQTmY2Ch,$Outh $Ide $Uicide,High Tide in the Snake's Nest,$uicideboy$,2015-06-15,169319,74,0.797,0.005380,0.516,...,0.0728,-6.286,0.0832,0.0478,1,1,109.993,audio_features,4,9
2,0kEZlJh4mK1QRfb3CT5LPk,Kill Yourself (Part III),My Liver Will Handle What My Heart Can't,$uicideboy$,2015-09-21,145078,81,0.753,0.326000,0.680,...,0.4230,-7.092,0.0349,0.1920,5,0,105.977,audio_features,4,1
3,1hSBVYwG0UPeou0ORftrxm,Do You Believe in God?,Grey Sheep II,$uicideboy$,2016-05-25,118073,67,0.794,0.038800,0.535,...,0.5550,-5.698,0.0506,0.0487,0,1,120.067,audio_features,4,4
4,5UGAXwbA17bUC0K9uquGY2,Antarctica,Antarctica,$uicideboy$,2016-01-26,126850,78,0.715,0.553000,0.633,...,0.0905,-6.869,0.0804,0.3190,2,1,105.945,audio_features,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47309,6cGBPNNRb4mt8m3uD9g1w8,Dagon,Psych Funk 101,Various Artists,2009-09-01,597013,6,0.369,0.505000,0.859,...,0.3290,-8.210,0.2020,0.0362,7,0,125.811,audio_features,4,9
47310,4zUU0VOduQFLb5iQhmaNDD,Chant for Ezkaton 2000 E. V.,The Apostasy,Behemoth,2007-07-17,310226,38,0.505,0.000074,0.989,...,0.6980,-6.861,0.0859,0.0536,3,1,90.038,audio_features,4,4
47311,2H3g6Q8T8g8Dvr0L15ntyv,Chant for Ezkaton 2000 E.V.,In Absentia Dei,Behemoth,2021-12-17,338000,28,0.447,0.000213,0.978,...,0.3120,-6.984,0.0959,0.0455,10,0,90.111,audio_features,4,0
47312,4gxMc88pEX4OnDG1sBW5R3,ESCALON (SHINJI'S THEME) [BLEACH],ESCALON (SHINJI'S THEME) [BLEACH],Kill AZYN,2021-12-19,113903,43,0.375,0.006380,0.767,...,0.1890,-9.303,0.0946,0.6160,7,1,157.725,audio_features,5,9


In [21]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9d897a87744a49638d588daeca7f0994",
                                                           client_secret="1e0c19e2db6c4e3182c30885a6491ffd"))

In [22]:
song_df = load_songs("./data/song_db.csv")

C:\Users\roman\AppData\Local\Temp/ipykernel_11580/701837643.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  song_df = load_songs("./data/song_db.csv")
b'Skipping line 3335: expected 20 fields, saw 21\nSkipping line 3336: expected 20 fields, saw 21\nSkipping line 3343: expected 20 fields, saw 21\nSkipping line 3345: expected 20 fields, saw 21\nSkipping line 3350: expected 20 fields, saw 21\nSkipping line 4630: expected 20 fields, saw 22\nSkipping line 4632: expected 20 fields, saw 22\nSkipping line 4634: expected 20 fields, saw 22\nSkipping line 4643: expected 20 fields, saw 22\nSkipping line 4644: expected 20 fields, saw 22\nSkipping line 4645: expected 20 fields, saw 22\nSkipping line 4646: expected 20 fields, saw 22\nSkipping line 4647: expected 20 fields, saw 22\nSkipping line 4650: expected 20 fields, saw 22\nSkipping line 4651: expected 20 fields, saw 22\nSkipping line 4655: expected 20 fields, saw 22\nSkipping lin

In [52]:
while True:
    song_array = user_input_get_features(song_df)
    get_random_song_cluster(song_df, song_array)

What song are you looking for? do you feel it
Song not found in local database, searching on Spotify.
Following artists found for "do you feel it":
 
 + 1: Chaos Chaos
 + 2: Panic! At The Disco
 + 3: Holy Wave
 + 4: Gwen Stefani
 + 5: The Blue Notes
 + 6: Slow Pulp
 + 7: Ella Mai
 + 8: Goldroom
 + 9: Summer Walker
 + 10: Joe Cuba Sextet
 + 11: Mike W3lts
 + 12: YoungBoy Never Broke Again
 + 13: corby
 + 14: Omar Apollo
 + 15: Joe Cuba
Please specify your entry (1 to 15): 1


C:\Users\roman\Python\anaconda\envs\DA_Env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


cluster: 5
get_r 2nZ3DQaL3Q9AB4UcQCE36b
Bubble Butt by Major Lazer
What song are you looking for? Safe from Harm
Song not found in local database, searching on Spotify.
Following artists found for "Safe from Harm":
 
 + 1: Massive Attack
 + 2: Chris Stapleton
 + 3: 4 Strings
 + 4: Whitney Houston
 + 5: 4 Strings
 + 6: The Magician
 + 7: 4 Strings
 + 8: Sufjan Stevens
 + 9: Bebe Winans
 + 10: Sufjan Stevens
 + 11: Markus Schulz
 + 12: The Magician
 + 13: Markus Schulz
 + 14: The Magician
 + 15: Massive Attack
Please specify your entry (1 to 15): 1


C:\Users\roman\Python\anaconda\envs\DA_Env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


cluster: 5
get_r 6ig2r8kFzCLLRbF6t7cli3
Cell Block Tango by Catherine Zeta-Jones


KeyboardInterrupt: Interrupted by user

In [ ]:
# track_id = "4yzlbUDqrP1ZriRp7zAcIP"
# IFrame(src="https://open.spotify.com/embed/track/"+track_id,
#        width="320",
#        height="80",
#        frameborder="0",
#        allowtransparency="true",
#        allow="encrypted-media",
#       )